<a href="https://colab.research.google.com/github/Ahomagai/MIMICIII_frequent_user_pred/blob/main/RandomForest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Bagging Classifier time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler

%matplotlib inline

In [3]:
from google.colab import files


uploaded = files.upload()

Saving mimic_v6_labelled.csv to mimic_v6_labelled.csv


In [4]:
#import libraries and data
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
import pandas as pd

In [5]:
df = pd.read_csv("/content/mimic_v6_labelled.csv")

print(df.shape)

(26176, 28)


In [6]:
# Identify and replace erroneous temp values
# Sort the DataFrame by subject_id and any other relevant column (e.g., timestamp)
df.sort_values(by=['subject_id', 'admittime'], inplace=True)

# Create a function to replace the values of 6 with the last value above 30
def replace_values(group):
    last_above_30 = None
    new_temp = []

    for temp in group['temp']:
        if temp > 30:
            last_above_30 = temp
        elif last_above_30 is None:
            last_above_30 = 36.66667  # Default value if no values above 30
        new_temp.append(last_above_30 if temp in (4, 5, 6) else temp)

    group['temp'] = new_temp
    return group


# Apply the function to each group of unique participants
df = df.groupby('subject_id').apply(replace_values)

# Reset the index of the DataFrame
df.reset_index(drop=True, inplace=True)

<ipython-input-6-63ef52bf1172>:22: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  df = df.groupby('subject_id').apply(replace_values)


In [7]:
# clean df, remove unrequired features and setup categorical and numerical df
df['Medication_count'] = df['unique_cv_items_count'] + df['unique_mv_items_count']


bin_edges = [0, 1, 2, 3, 4, float('inf')]
bin_labels = ['1', '2', '3', '4', '>4']

# Apply binning to the 'num_unique_icd9_codes' column
df['num_unique_icd9_codes_bin'] = pd.cut(df['num_unique_icd9_codes'], bins=bin_edges, labels=bin_labels)

# Apply binning to the 'Medication_count' column
df['Medication_count_bin'] = pd.cut(df['Medication_count'], bins=bin_edges, labels=bin_labels)

df = df.drop(columns=['Unnamed: 0.1','Unnamed: 0','subject_id','admittime',
                      'deathtime','marital_status',
                      'hadm_id', 'icd9_code_2','icd9_code_3',
                      'dischtime','admission_type',
                      'unique_cv_items_count','unique_mv_items_count',
                      'num_unique_icd9_codes','Medication_count','diagnosis'])

df= df.dropna()

In [8]:
# replace Age > 300
def replace_above_300(value):
    if value > 300:
        return value - 210
    return value

# Apply the function to the 'age' column
df['age'] = df['age'].apply(replace_above_300)

In [9]:
df.describe()

,age,los,icd9_code_1,hr,nbp,spo2,rr,temp,Label
count,20883.000000,20883.000000,20883.000000,20883.000000,20883.000000,20883.000000,20883.000000,20883.000000,20883.000000
mean,63.304587,10.967243,4879.559690,88.512580,87.774858,97.259015,18.516545,33.622603,0.121391
std,17.304137,10.737337,3103.902339,20.586571,832.051419,4.633746,6.203092,10.019697,0.326588
min,18.022083,-0.116667,11.000000,0.000000,0.000000,0.000000,0.000000,0.777778,0.000000
25%,51.942804,4.739583,3601.000000,75.000000,69.000000,96.000000,14.000000,36.444401,0.000000
50%,64.843536,7.865972,3951.000000,87.000000,80.000000,98.000000,18.000000,36.666670,0.000000
75%,77.225612,13.505556,8102.000000,100.000000,92.333298,100.000000,22.000000,36.666670,0.000000
max,100.280859,202.762500,9978.000000,941.000000,120130.030000,100.000000,160.000000,40.833302,1.000000


In [10]:
df.keys()

Index(['age', 'gender', 'los', 'discharge_location', 'insurance', 'ethnicity',
       'icd9_code_1', 'hr', 'nbp', 'spo2', 'rr', 'temp', 'Label',
       'num_unique_icd9_codes_bin', 'Medication_count_bin'],
      dtype='object')

In [11]:
categ_columns = ['gender','discharge_location','insurance','ethnicity',
                      'icd9_code_1','num_unique_icd9_codes_bin','Medication_count_bin']

from sklearn.preprocessing import LabelEncoder

label_encoders = {}
for column in categ_columns:
  label_encoder = LabelEncoder()
  df[column] = label_encoder.fit_transform(df[column])
  label_encoders[column] = label_encoder


In [12]:
# split into X and y
y_df = df['Label']
y_df.columns = ['Label']

In [13]:
# Split X numerical and Categorical
x_df = df.copy()

#x_df_cat = x_df[['gender','discharge_location','insurance','ethnicity',
               # 'icd9_code_1','num_unique_icd9_codes_bin','Medication_count_bin']].copy()

x_df_cat = pd.get_dummies(df[['discharge_location','insurance','ethnicity','icd9_code_1','gender', 'Medication_count_bin', 'num_unique_icd9_codes_bin']])

x_df_num = x_df.drop(['gender','discharge_location','insurance','ethnicity',
                      'icd9_code_1','Label','num_unique_icd9_codes_bin','Medication_count_bin'], axis = 1).astype('float64')
x_df_num

,age,los,hr,nbp,spo2,rr,temp
0,76.526788,10.784722,95.0,36.666698,98.0,16.0,36.055599
1,47.845044,7.759028,111.0,80.666702,98.0,18.0,37.444401
2,50.148292,25.529167,90.0,64.000000,96.0,19.0,37.222198
3,39.866116,6.855556,80.0,71.333298,98.0,19.0,37.277802
5,50.841554,2.206250,95.0,113.667000,100.0,27.0,37.666698
...,...,...,...,...,...,...,...
26171,65.262831,6.814583,71.0,66.000000,95.0,18.0,36.666670
26172,65.377831,6.128472,94.0,71.000000,99.0,21.0,36.666670
26173,78.576622,3.079861,69.0,82.000000,98.0,21.0,36.666670
26174,53.808478,16.595833,118.0,102.000000,84.0,33.0,36.666670


In [14]:
# numerical is under x_df_num
X_numerical = x_df_num

In [15]:
print(len(x_df), len(y_df), len(x_df_num))

20883 20883 20883


In [16]:
list_numerical = X_numerical.columns
list_numerical

Index(['age', 'los', 'hr', 'nbp', 'spo2', 'rr', 'temp'], dtype='object')

In [17]:
for key in x_df_cat.keys():
    print('CATEGORICAL KEYS', key)

for key in x_df_num.keys():
    print('\n','NUMERIC KEYS',key)

CATEGORICAL KEYS discharge_location
CATEGORICAL KEYS insurance
CATEGORICAL KEYS ethnicity
CATEGORICAL KEYS icd9_code_1
CATEGORICAL KEYS gender
CATEGORICAL KEYS Medication_count_bin
CATEGORICAL KEYS num_unique_icd9_codes_bin

 NUMERIC KEYS age

 NUMERIC KEYS los

 NUMERIC KEYS hr

 NUMERIC KEYS nbp

 NUMERIC KEYS spo2

 NUMERIC KEYS rr

 NUMERIC KEYS temp


In [18]:
# Scale numerical features
scaled_x_df_num = pd.DataFrame(StandardScaler().fit_transform(x_df_num), columns = x_df_num.keys())

In [19]:
scaled_x_df_num.describe()

,age,los,hr,nbp,spo2,rr,temp
count,2.088300e+04,2.088300e+04,2.088300e+04,2.088300e+04,2.088300e+04,2.088300e+04,2.088300e+04
mean,-5.335110e-16,-1.224898e-16,-1.742077e-16,-1.360997e-18,2.653945e-17,-2.041496e-16,6.532788e-16
std,1.000024e+00,1.000024e+00,1.000024e+00,1.000024e+00,1.000024e+00,1.000024e+00,1.000024e+00
min,-2.616923e+00,-1.032302e+00,-4.299633e+00,-1.054946e-01,-2.098979e+01,-2.985122e+00,-3.278104e+00
25%,-6.566092e-01,-5.800142e-01,-6.563941e-01,-2.256508e-02,-2.717121e-01,-7.281292e-01,2.816318e-01
50%,8.893741e-02,-2.888374e-01,-7.347585e-02,-9.344428e-03,1.599145e-01,-8.327410e-02,3.038155e-01
75%,8.045104e-01,2.364062e-01,5.580189e-01,5.478686e-03,5.915411e-01,5.615810e-01,3.038155e-01
max,2.136897e+00,1.786289e+01,4.141087e+01,1.442761e+02,5.915411e-01,2.280908e+01,7.196695e-01


In [20]:
# Find outliers based on 1.5*(Q3-Q1) outlier step
outliers  = []
# For each feature find the data points with extreme high or low values
for feature in scaled_x_df_num.keys():
    # Calculate Q1 (25th percentile of the data) for the given feature
    Q1 = np.percentile(scaled_x_df_num[feature],25)
    # Calculate Q3 (75th percentile of the data) for the given feature
    Q3 = np.percentile(scaled_x_df_num[feature],75)
    # Use the interquartile range to calculate an outlier step (1.5 times the interquartile range)
    step = 1.5*(Q3-Q1)
    outliers_per_feature = scaled_x_df_num[~((scaled_x_df_num[feature] >= Q1 - step) & (scaled_x_df_num[feature] <= Q3 + step))]
    outliers = outliers + list(outliers_per_feature.index)
outliers.sort()
outliers = list(set(outliers))

# Remove outliers
clean_scaled_x_df_num = scaled_x_df_num.drop(scaled_x_df_num.index[outliers]).reset_index(drop = True)
print('{} outliers removed'.format(len(outliers)))

8816 outliers removed


In [21]:
# Convert categorical features to "One Hot Encoding" vectors and drop outliers
clean_x_df_cat = x_df_cat.drop(x_df_cat.index[outliers]).reset_index(drop = True)
# clean_x_df_cat_ohe = pd.get_dummies(clean_x_df_cat, drop_first=True)
# dummies = pd.get_dummies(clean_x_df_cat)

In [22]:
# Recombine scaled numerical and encoded categorical features by concatination
x_df = pd.concat([clean_scaled_x_df_num, clean_x_df_cat], axis = 1)
# x_df = pd.concat([clean_scaled_x_df_num, dummies], axis = 1)

In [23]:
# Drop outliers from label Dataframe as well
y_df = y_df.drop(y_df.index[outliers]).reset_index(drop = True)

In [24]:
print(len(y_df), len(x_df))

12067 12067


In [25]:
y_df.columns = ['Label']

In [26]:
print(y_df.value_counts())
print('\nPositive label proportion: ')
print(y_df.value_counts()/y_df.shape[0])

0    10658
1     1409
Name: Label, dtype: int64

Positive label proportion: 
0    0.883235
1    0.116765
Name: Label, dtype: float64


In [27]:
# write final files to local dir
x_df.to_csv('/content/x_df.csv')
y_df.to_csv('/content/y_df.csv')

In [ ]:
# Bagging Classifier time

In [28]:
# Import data
y = pd.read_csv('/content/y_df.csv')
X = pd.read_csv('/content/x_df.csv')

In [29]:
# Remove unwanted column
X = X.drop(columns = ['Unnamed: 0'])
y = y.drop(columns = ['Unnamed: 0'])

print(X.keys())
print(y.keys())


Index(['age', 'los', 'hr', 'nbp', 'spo2', 'rr', 'temp', 'discharge_location',
       'insurance', 'ethnicity', 'icd9_code_1', 'gender',
       'Medication_count_bin', 'num_unique_icd9_codes_bin'],
      dtype='object')
Index(['Label'], dtype='object')


In [30]:
from sklearn.model_selection import cross_validate
from sklearn.svm import LinearSVC
from sklearn.ensemble import  RandomForestClassifier
import pandas as pd

In [31]:
# split into test & train
from sklearn.model_selection import train_test_split

test_size = 0.20
seed= 7

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=seed)

In [32]:
# balance training sets
from imblearn.combine import SMOTETomek
from imblearn.under_sampling import TomekLinks

resample = SMOTETomek(tomek=TomekLinks(sampling_strategy='majority'))
X_train,y_train = resample.fit_resample(X_train,y_train)

In [33]:
clf=RandomForestClassifier(random_state = 1, class_weight="balanced")
output = cross_validate(clf, X, y, scoring = ['accuracy','balanced_accuracy','average_precision','f1'], return_estimator =True)

/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:686: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:686: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:686: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:686: DataConversionWarning: A column-vector y was passed whe

In [34]:
print('K-fold (5) Cross Validation Metrics- MEAN (MIN | MAX):' '\n',
      'Test accuracy: ', output['test_accuracy'].mean(),'(',output['test_accuracy'].min(),'|',output['test_accuracy'].max(),')', '\n',
      'Test balanced accuracy: ',output['test_balanced_accuracy'].mean(),'(',output['test_balanced_accuracy'].min(),'|',output['test_balanced_accuracy'].max(),')','\n',
      'Test average precision: ',output['test_average_precision'].mean(),'(',output['test_average_precision'].min(),'|',output['test_average_precision'].max(),')','\n',
      'Test average f1 score: ' ,output['test_f1'].mean(),'(',output['test_f1'].min(),'|',output['test_f1'].max(),')')

K-fold (5) Cross Validation Metrics- MEAN (MIN | MAX):
 Test accuracy:  0.8830694069097553 ( 0.8823041856610029 | 0.884009942004971 ) 
 Test balanced accuracy:  0.5026767158096914 ( 0.4997653683716565 | 0.5066231554295902 ) 
 Test average precision:  0.22360946051746994 ( 0.18807440295099428 | 0.24649761078692056 ) 
 Test average f1 score:  0.012548630944009634 ( 0.0 | 0.027777777777777776 )


In [37]:
model = clf.fit(X_train, y_train)
score = model.score(X_test, y_test)
print(score)

output = cross_validate(model, X, y, scoring = ['accuracy','balanced_accuracy','average_precision','f1'], return_estimator =True)

<ipython-input-37-4291c9b86575>:1: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model = clf.fit(X_train, y_train)


0.8388566694283347


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:686: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:686: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:686: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:686: DataConversionWarning: A column-vector y was passed whe

In [38]:
print('K-fold (5) Cross Validation Metrics- MEAN (MIN | MAX):' '\n',
      'Test accuracy: ', output['test_accuracy'].mean(),'(',output['test_accuracy'].min(),'|',output['test_accuracy'].max(),')', '\n',
      'Test balanced accuracy: ',output['test_balanced_accuracy'].mean(),'(',output['test_balanced_accuracy'].min(),'|',output['test_balanced_accuracy'].max(),')','\n',
      'Test average precision: ',output['test_average_precision'].mean(),'(',output['test_average_precision'].min(),'|',output['test_average_precision'].max(),')','\n',
      'Test average f1 score: ' ,output['test_f1'].mean(),'(',output['test_f1'].min(),'|',output['test_f1'].max(),')')

K-fold (5) Cross Validation Metrics- MEAN (MIN | MAX):
 Test accuracy:  0.8830694069097553 ( 0.8823041856610029 | 0.884009942004971 ) 
 Test balanced accuracy:  0.5026767158096914 ( 0.4997653683716565 | 0.5066231554295902 ) 
 Test average precision:  0.22360946051746994 ( 0.18807440295099428 | 0.24649761078692056 ) 
 Test average f1 score:  0.012548630944009634 ( 0.0 | 0.027777777777777776 )


In [36]:
for idx,estimator in enumerate(output['estimator']):
    print("Features sorted by their score for estimator {}:".format(idx))
    feature_importances = pd.DataFrame(estimator.feature_importances_,
                                       index = X.columns,
                                        columns=['importance']).sort_values('importance', ascending=False)
    print(feature_importances)

Features sorted by their score for estimator 0:
                           importance
age                          0.152385
icd9_code_1                  0.126719
los                          0.119284
nbp                          0.110768
hr                           0.105991
rr                           0.084088
spo2                         0.059027
discharge_location           0.057453
ethnicity                    0.051257
num_unique_icd9_codes_bin    0.045003
insurance                    0.031931
temp                         0.024494
gender                       0.018737
Medication_count_bin         0.012864
Features sorted by their score for estimator 1:
                           importance
age                          0.142966
icd9_code_1                  0.123488
los                          0.116990
nbp                          0.110222
hr                           0.106470
rr                           0.083073
spo2                         0.057258
discharge_location           0